In [10]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import requests
import sqlalchemy
from functools import reduce 
from fake_useragent import UserAgent



In [11]:
import sqlite3
import sys

In [17]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///hrefs.db")
hrefs = pd.read_sql('''
SELECT * FROM hrefs;''', engine)
engine = create_engine("sqlite:///courses_2.db")
courses = pd.read_sql('''
SELECT * FROM courses;''', engine)

hrefs

,href
0,/course-details/10008071/AAUNDERGRADUATE5YEAR/...
1,/course-details/10000047/PSSFDOPTDIS/Full-time
2,/course-details/10007783/NN24/Full-time
3,/course-details/10007783/NL41/Full-time
4,/course-details/10007783/NN34/Full-time
...,...
35208,/course-details/10007713/U1BATODMW/Full-time
35209,/course-details/10007713/U1BATODMS/Full-time
35210,/course-details/10007713/U1BAWSHIJF/Full-time
35211,/course-details/10007713/U1BAWSHIJ/Full-time


In [13]:
course_columns = [
    'href',#0
    
    'study_mode',#1
    'study_length',
    'distance_learning',
    'placement_year',
    'abroad_year',
    'foundation_year',
    
    'satisfaction',#7
    
    'teaching_1',#8
    'teaching_2',
    'teaching_3',
    'teaching_4',
    'opportunities_1',
    'opportunities_2',
    'opportunities_3',
    'feedback_1',
    'feedback_2',
    'feedback_3',
    'feedback_4',
    'support_1',
    'support_2',
    'support_3',
    'management_1',
    'management_2',
    'management_3',
    'resources_1',
    'resources_2',
    'resources_3',
    'community_1',
    'community_2',
    'voice_1',
    'voice_2',
    'voice_3',
    'voice_4',#33
    
    'earnings_15m',#34
    'earnings_3y',
    'earnings_5y',
    
    'usefulness',#37
    'meaningfulness',
    'future',

]

In [85]:
# SCRAPE SINGLE
href = '/course-details/10001883/422NDM/Full-time/'
url = 'https://discoveruni.gov.uk{}'.format(href)
page = requests.get(url, headers = user_agent).text
soup = BeautifulSoup(page, "html5lib")

data = [href]+[np.nan]*39
try:
    data[1:7] = [x.find_next().text.strip() for x in soup.find_all('p', class_='course-detail__overview-item-heading')][:6]
except:
    data[1:7] = [np.nan]*6
    
single_chart_regex = re.compile('^(student-satisfaction__chart discover)')
earnings_section_regex = re.compile('^(earnings-after-course_section-)')
earnings_regex = re.compile('^(earnings-after-course-stat-salary)')


satisfaction_tab_regex = re.compile('^(student-satisfaction-[1-9])')
perceptions_tab_regex = re.compile('^(grad-perceptions-[1-9])')
earnings_tab_regex = re.compile('^(earnings-after-course-[1-9])')

satisfaction_tabs = soup.find_all('div', {"id": satisfaction_tab_regex})
perceptions_tabs = soup.find_all('div', {"id": perceptions_tab_regex})
earnings_tabs = soup.find_all('div', {"id": earnings_tab_regex})

for subject in range(len(satisfaction_tabs)):
    try:
        data[7] = int(satisfaction_tabs[subject].find('p', class_='student-satisfaction__overview-lead text-center').text.strip()[:-1])
        data[8:34] = [int(x.get('data-value')) for x in satisfaction_tabs[subject].find_all('div', class_=single_chart_regex)][:26]
    except:
        data[7:34] = [np.nan]*27
        
    try:
        data[34:37] = [x.find('h2', class_=earnings_regex).text.replace(',','').replace('£','') for x in 
                     earnings_tabs[subject].find('div', class_='discover-uni-container').find_all('div', class_=earnings_section_regex)]
    except:
        data[34:37] = [np.nan]*3

    try:
        temp = [int(x.get('data-value')) for x in perceptions_tabs[subject].find_all('div', class_=single_chart_regex)]
        if len(temp)==3:
            data[37:] = temp
    except:
        data[37:] = [np.nan]*3
        

# else:
#     pd.DataFrame([data]).to_sql('courses', courses_connection, if_exists='append', index=False)

pd.DataFrame([data],columns=course_columns)



,href,study_mode,study_length,distance_learning,placement_year,abroad_year,foundation_year,satisfaction,teaching_1,teaching_2,...,voice_1,voice_2,voice_3,voice_4,earnings_15m,earnings_3y,earnings_5y,usefulness,meaningfulness,future
0,/course-details/10001883/422NDM/Full-time/,Full time,2 year course,Not Available,Not Available,Not Available,Not Available,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,,17000,23500,NaN,NaN,NaN


In [ ]:
8358

In [14]:
# SCRAPE ALL

courses_connection = sqlite3.connect('courses_2.db')
ua = UserAgent()

for href in hrefs['href'][32294:]:    
    url = 'https://discoveruni.gov.uk{}'.format(href)
    user_agent = {'User-agent': ua.random}
    page = requests.get(url,headers = user_agent).text
    soup = BeautifulSoup(page, "html5lib")

    data = [href]+[np.nan]*39
    try:
        data[1:7] = [x.find_next().text.strip() for x in soup.find_all('p', class_='course-detail__overview-item-heading')][:6]
    except:
        data[1:7] = [np.nan]*6

    single_chart_regex = re.compile('^(student-satisfaction__chart discover)')
    earnings_section_regex = re.compile('^(earnings-after-course_section-)')
    earnings_regex = re.compile('^(earnings-after-course-stat-salary)')


    satisfaction_tab_regex = re.compile('^(student-satisfaction-[1-9])')
    perceptions_tab_regex = re.compile('^(grad-perceptions-[1-9])')
    earnings_tab_regex = re.compile('^(earnings-after-course-[1-9])')

    satisfaction_tabs = soup.find_all('div', {"id": satisfaction_tab_regex})
    perceptions_tabs = soup.find_all('div', {"id": perceptions_tab_regex})
    earnings_tabs = soup.find_all('div', {"id": earnings_tab_regex})

    for subject in range(len(satisfaction_tabs)):
        try:
            data[7] = int(satisfaction_tabs[subject].find('p', class_='student-satisfaction__overview-lead text-center').text.strip()[:-1])
            data[8:34] = [int(x.get('data-value')) for x in satisfaction_tabs[subject].find_all('div', class_=single_chart_regex)][:26]
        except:
            data[7:34] = [np.nan]*27

        try:
            data[34:37] = [x.find('h2', class_=earnings_regex).text.replace(',','').replace('£','') for x in 
                         earnings_tabs[subject].find('div', class_='discover-uni-container').find_all('div', class_=earnings_section_regex)]
        except:
            data[34:37] = [np.nan]*3

        try:
            temp = [int(x.get('data-value')) for x in perceptions_tabs[subject].find_all('div', class_=single_chart_regex)]
            if len(temp)==3:
                data[37:] = temp
        except:
            data[37:] = [np.nan]*3
        pd.DataFrame([data],columns=course_columns).to_sql('courses', courses_connection, if_exists='append', index=False)

courses_connection.close()

In [18]:
# SCRAPE ALL MISSING

courses_connection = sqlite3.connect('courses_2.db')
ua = UserAgent()

for href in hrefs['href'][~hrefs['href'].isin(courses['href'].drop_duplicates())]:    
    url = 'https://discoveruni.gov.uk{}'.format(href)
    user_agent = {'User-agent': ua.random}
    page = requests.get(url,headers = user_agent).text
    soup = BeautifulSoup(page, "html5lib")

    data = [href]+[np.nan]*39
    try:
        data[1:7] = [x.find_next().text.strip() for x in soup.find_all('p', class_='course-detail__overview-item-heading')][:6]
    except:
        data[1:7] = [np.nan]*6

    single_chart_regex = re.compile('^(student-satisfaction__chart discover)')
    earnings_section_regex = re.compile('^(earnings-after-course_section-)')
    earnings_regex = re.compile('^(earnings-after-course-stat-salary)')


    satisfaction_tab_regex = re.compile('^(student-satisfaction-[1-9])')
    perceptions_tab_regex = re.compile('^(grad-perceptions-[1-9])')
    earnings_tab_regex = re.compile('^(earnings-after-course-[1-9])')

    satisfaction_tabs = soup.find_all('div', {"id": satisfaction_tab_regex})
    perceptions_tabs = soup.find_all('div', {"id": perceptions_tab_regex})
    earnings_tabs = soup.find_all('div', {"id": earnings_tab_regex})

    for subject in range(len(satisfaction_tabs)):
        try:
            data[7] = int(satisfaction_tabs[subject].find('p', class_='student-satisfaction__overview-lead text-center').text.strip()[:-1])
            data[8:34] = [int(x.get('data-value')) for x in satisfaction_tabs[subject].find_all('div', class_=single_chart_regex)][:26]
        except:
            data[7:34] = [np.nan]*27

        try:
            data[34:37] = [x.find('h2', class_=earnings_regex).text.replace(',','').replace('£','') for x in 
                         earnings_tabs[subject].find('div', class_='discover-uni-container').find_all('div', class_=earnings_section_regex)]
        except:
            data[34:37] = [np.nan]*3

        try:
            temp = [int(x.get('data-value')) for x in perceptions_tabs[subject].find_all('div', class_=single_chart_regex)]
            if len(temp)==3:
                data[37:] = temp
        except:
            data[37:] = [np.nan]*3
        pd.DataFrame([data],columns=course_columns).to_sql('courses', courses_connection, if_exists='append', index=False)

courses_connection.close()

In [ ]:
courses_connection.close()